<a href="https://colab.research.google.com/github/pdevis/punpy/blob/master/esa_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Example using digital effects tables**

In this notebook, we will show how digital effects tables, created with obsarray (see xx), can be propagated through a measurement function using punpy. Here we use an example where we calculate the uncertainties in a volume of gas, using the ideal gas law and a digital effects table quantifying the uncertainties and error-correlation of the gas temperature, pressure and amount of substance (number of moles).

We first install and import the obsarray and punpy packages (and xarray, numpy and matplotlib), and read in the digital effects table created in this notebook:

In [ ]:
!pip install punpy==0.25!
!pip install obsarray==0.25!

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import obsarray
import punpy

ds = xr.open_dataset("digital_effects_table_gaslaw_example.nc")  # read digital effects table


Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

This digital effects table has multiple uncertainty components for each of the included variables. The input quantities and their uncertainties have thus been fully defined within this obsarray object. The only thing that remains to be done before we can propagate the uncertainties is to define the measurement function. In order to be able to use the digital effects tables, this measurement function needs to be defined by subclassing the MeasurementFunction imported from punpy, and then xx the function called meas_function. Here we make a measurement function that implements the ideal gas law:

In [ ]:
from punpy import MeasurementFunction

# Define your measurement function inside a subclass of MeasurementFunction
class IdealGasLaw(MeasurementFunction):
    def meas_function(self, pres, temp, n):
        return (n *temp * 8.134)/pres

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

Once we have defined the IdealGasLaw, we can create an object of this class. The initialiser of the MeasurementFunction class needs a number of arguments. The first argument is a list with the names of the variables in the digital effects table dataset (ds) that are used in the meas_function (and in the order that they are used in the meas_function). The second argument is whether the Monte Carlo (MC) or Law of Propagation of Uncertainty (LPU) method is used. Finally, there are a number of optional arguments relevant to the MC or LPU methods, or which provide additional functionality (see later). Here, we set the number of MC steps to 100000.

In [ ]:
gl = IdealGasLaw(["pressure", "temperature", "n_moles"], "MC", steps=100000)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

We can now propagate the uncertainties using a single line of code with the propagate_ds function. The output will be a digital effects table dataset with the measurand and the combined random uncertainties, the combined systematic uncertainties and the combined structured uncertainties on the measurand. The propagate_ds function needs to be provided the name of the measurand as the first argument, and then the digital effects table (or multiple digital effects tables) as the subsequent arguments.

In [ ]:
ds_y = gl.propagate_ds("Volume", ds)
print(ds_y)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

and make the plots for the L1 data:

In [ ]:
make_plots_L1(L1,L1_ur,L1_us,L1_ut,L1_corr)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

**Correlated errors**

In addition to propagating random (uncorrelated) and systematic (fully correlated) uncertainties it is also possible to propagate uncertainties associated with structured errors. If we know the covariance matrix for each of the input quantities, it is straigtforward to propagate these. In the below example we assume the L0 data and dark data to be uncorrelated (their covariance matrix is a, diagonal matrix) and gains to be a custom covariance:

In [ ]:
# your uncertainties
L0_ur = L0*0.05  # 5% random uncertainty
dark_ur = np.array([0.01,0.002,0.006,0.002,0.015])  # random uncertainty

L0_cov=punpy.convert_corr_to_cov(np.eye(len(L0_ur)),L0_ur)
dark_cov=punpy.convert_corr_to_cov(np.eye(len(dark_ur)),dark_ur )
gains_cov= np.array([[0.45,0.35,0.30,0.20,0.05],
                    [0.35,0.57,0.32,0.30,0.07],
                    [0.30,0.32,0.56,0.24,0.06],
                    [0.20,0.30,0.24,0.44,0.04],
                    [0.05,0.07,0.06,0.04,0.21]])


Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

In [ ]:
prop=punpy.MCPropagation(10000)
L1=calibrate(L0,gains,dark)
L1_ut,L1_corr=prop.propagate_cov(calibrate,[L0,gains,dark],
                                 [L0_cov,gains_cov,dark_cov],return_corr=True)

make_plots_L1(L1,L1_ut=L1_ut,L1_corr=L1_corr)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

In addition to having a correlation along one or more dimensions of a given variable, it is also possible two variables are correlated. This can be specified in punpy by using the corr_between keyword. In the example below, the systematic errors in the darks and L0 data are fully correlated.

In [ ]:
prop=punpy.MCPropagation(10000)
L1=calibrate(L0,gains,dark)

corr_var=np.array([[1,0,1],[0,1,0],[1,0,1]])

L1_ur=prop.propagate_random(calibrate,[L0,gains,dark],
      [L0_ur,gains_ur,dark_ur])
L1_us=prop.propagate_systematic(calibrate,[L0,gains,dark],
      [L0_us,gains_us,L0_us],corr_between=corr_var)
L1_ut=(L1_ur**2+L1_us**2)**0.5
L1_cov=punpy.convert_corr_to_cov(np.eye(len(L1_ur)),L1_ur)+\
       punpy.convert_corr_to_cov(np.ones((len(L1_us),len(L1_us))),L1_us)
L1_corr=punpy.correlation_from_covariance(L1_cov)
make_plots_L1(L1,L1_ur,L1_us,L1_ut,L1_corr)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

The above results were generated using a Monte Carlo Method. The law of propagation of uncertainty can also be used to propagate the uncertainties. In this case, the Jacobian is used in the propagation. The Jacobian can be specified manually, or if not will be calculated through numerical differentiation. 

In [ ]:
prop=punpy.LPUPropagation(parallel_cores=2)
L1=calibrate(L0,gains,dark)

def J_calibrate(L0,gains,dark):
    Jac_x1 = np.diag(gains)
    Jac_x2 = np.diag(L0-dark)
    Jac_x3 = np.diag(-gains)
    Jac = np.concatenate((Jac_x1, Jac_x2, Jac_x3)).T
    return Jac
    
Jx=J_calibrate(L0,gains,dark)
L1_ur=prop.propagate_random(calibrate,[L0,gains,dark],
      [L0_ur,gains_ur,dark_ur],Jx=Jx)
L1_us=prop.propagate_systematic(calibrate,[L0,gains,dark],
      [L0_us,gains_us,np.zeros(5)],Jx=Jx,corr_between=corr_var)
L1_ut=(L1_ur**2+L1_us**2)**0.5
L1_cov=punpy.convert_corr_to_cov(np.eye(len(L1_ur)),L1_ur)+\
       punpy.convert_corr_to_cov(np.ones((len(L1_us),len(L1_us))),L1_us)
L1_corr=punpy.correlation_from_covariance(L1_cov)
make_plots_L1(L1,L1_ur,L1_us,L1_ut,L1_corr)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

**punpy for data with more dimensions**

In reality, propagation of uncertainty in EO is applied to larger datasets with higher dimensionality. Instead of the above 5 datapoints, we might have 5 wavelengths but 100 by 50 pixel images for each of these wavelengths. These can offcourse also be handled by punpy.

In [ ]:
# your data
wavs = np.array([350,450,550,650,750])

L0 = np.tile([0.43,0.8,0.7,0.65,0.9],(50,100,1)).T
L0 = L0 + np.random.normal(0.0,0.05,L0.shape)

dark = np.tile([0.05,0.03,0.04,0.05,0.06],(50,100,1)).T
gains = np.tile([23,26,28,29,31],(50,100,1)).T

# your uncertainties
L0_ur = L0*0.05  # 5% random uncertainty
L0_us = np.ones((5,100,50))*0.03  # systematic uncertainty of 0.03
                         # (common between bands)

gains_ur = np.tile(np.array([0.5,0.7,0.6,0.4,0.1]),(50,100,1)).T  # random uncertainty
gains_us = np.tile(np.array([0.1,0.2,0.1,0.4,0.3]),(50,100,1)).T  # systematic uncertainty
# (different for each band but fully correlated)
dark_ur = np.tile(np.array([0.01,0.002,0.006,0.002,0.015]),(50,100,1)).T  # random uncertainty

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

In [ ]:
prop=punpy.MCPropagation(1000,)
L1=calibrate(L0,gains,dark)
L1_ur=prop.propagate_random(calibrate,[L0,gains,dark],
      [L0_ur,gains_ur,dark_ur],repeat_dims=[1])
L1_us=prop.propagate_systematic(calibrate,[L0,gains,dark],
      [L0_us,gains_us,None],repeat_dims=[1])
L1_ut=(L1_ur**2+L1_us**2)**0.5


Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

We then define a new function to plot images of the relative uncertainties in each band:

In [ ]:
def make_plots_L1_image(wavs,L1,L1_u=None,c_range=[0,0.1]):
  fig, axs = plt.subplots(1,len(wavs),figsize=(20,5))
  
  for i,ax in enumerate(axs):
    ax.set_xlabel("x_pix")
    ax.set_ylabel("y_pix")
    ax.set_title("%s nm rel uncertainties"%(wavs[i]))
    im_plot=ax.imshow(L1_u[i]/L1[i],vmin=c_range[0],vmax=c_range[1])

  plt.colorbar(im_plot)
  plt.show()

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

In [ ]:
make_plots_L1_image(wavs,L1,L1_ur)
make_plots_L1_image(wavs,L1,L1_us)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

For multidimensional input quantities, it is often the case that a certain correlation structure is known along one of the dimensions, and that the other dimensions are either completely independent (random) or fully correlated (systematic). For example below, we know the correlation structure for the systematic uncertainties on the gains wrt wavelength, and consider each of the measurements to be fully correlted wrt the spatial dimensions.

In [ ]:
gains_corr=np.array([[1.,0.14123392,0.12198785,0.07234254,0.01968095],
 [0.14123392,1.,0.1350783,0.12524757,0.0095603 ],
 [0.12198785,0.1350783,1.,0.1041107,0.02890266],
 [0.07234254,0.12524757,0.1041107,1.,0.01041678],
 [0.01968095,0.0095603,0.02890266,0.01041678,1.]])

L1_us,L1_us_corr=prop.propagate_systematic(calibrate,[L0,gains,dark],
      [None,gains_us,None],repeat_dims=[1,2],corr_x=[None,gains_corr,None],return_corr=True)

make_plots_L1_image(wavs,L1,L1_us)
make_plots_L1(np.mean(L1,axis=(1,2)),L1_us=np.mean(L1_us,axis=(1,2)),L1_corr=L1_us_corr)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).

In this case, the returned correlation matrix is again wrt wavelength, and the correlation structure of the repeated measurements is the same as it was in the inputs. In the above example, the uncertainties on the L0 and darks are set to None, and are thus not included. However, it is possible to include these, even if they have a different correlation structure than the uncertainties on the gains. In the example below, we repeat the same, but now include systematic uncertainties on the L0, that are fully correlated. It can be seem in this case we can just set corr_x to None, in which case it will default to a full correlation (because we are using the propagate_systematic function). If we were using propagate_random, it would default to independent errors.

In [ ]:
L1_us,L1_us_corr=prop.propagate_systematic(calibrate,[L0,gains,dark],
      [L0_us,gains_us,None],repeat_dims=[1,2],corr_x=[None,gains_corr,None],return_corr=True)

make_plots_L1_image(wavs,L1,L1_us)
make_plots_L1(np.mean(L1,axis=(1,2)),L1_us=np.mean(L1_us,axis=(1,2)),L1_corr=L1_us_corr)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.7.8 64-bit ('NPLtools': conda).